# Pokročilé dolování z databází - Průběžná zpráva
### Dan Balarin, Lukáš Vala
### Datasety: <a href = "https://archive-beta.ics.uci.edu/dataset/2/adult">Adult</a>, <a href="https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=1994&most_recent_year_desc=true&start=1993">GDP in 1994</a>

# Vypracování návodu pro zpracování a analýzu dat pomocí systému CleverMiner
## Krok 0 Instalace potřebných knihoven
- Clever Miner: `pip install cleverminer`
- pandas: `pip install pandas`
## Krok 1 Načtení a zpracování dat
Prvním krokem je načíst data pomocí knihovny pandas. Data se v tomto případě nacházejí ve složce data:

In [10]:
import pandas as pd

adult = pd.read_csv ('./data/adult.data', encoding='cp1250', sep=', ')
gdp = pd.read_csv ('./data/gdpStates.txt', encoding='cp1250', sep='\t')

print(adult.head(5))
print(gdp.head(5))

C:\Users\lukas\AppData\Local\Temp\ipykernel_3112\2916565987.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  adult = pd.read_csv ('./data/adult.data', encoding='cp1250', sep=', ')


   Age         Workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week        country income  
0          2174             0              40  United-States  <=50K  
1             0             0             

Pomocí parametru `sep` je důležité specifikovat, jaký separátor jednotlivých atributů je v tabulce použit. U datasetu adult je to `, ` a u GDP je to tabulátor. Správnost načtení dat můžeme poté zkontrolovat pomocí výpisu prvních 5 hodnot obou tabulek.

Dále bychom si také měli zkontrolovat, zda pandas správně určila datové typy u jednotlivých sloupců. 
To můžeme zjistit pomocí příkazu `dtypes`:

In [11]:
print(adult.dtypes)
print(gdp.dtypes)

Age                int64
Workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
income            object
dtype: object
country              object
Most Recent Year      int64
GDP per Capita      float64
dtype: object


Pro string vypíše pandas `object`, pro integer `int64` a pro desetinné číslo `float64`

## Krok 2 - Spojení dat
Nyní je třeba spojit 2 tabulky do sebe. Obě obsahují sloupec country. V tabulce `adult` se tento sloupec jmenuje `native-country`, je proto nutné ho přejmenovat přímo v datasetu na `country`, aby se shodoval s názvem sloupce ve druhém datasetu. Dále je potřeba upravit jednotlivé hodnoty v jedné z tabulek, protože např. hodnota `United States` v tabulce adult je původně zapsána v gdp tabulce jako `United-States`. Použijeme tedy funkci definovanou v ukázce pod tímto textem a následně v datech GDP najdeme jednotlivé hodnoty a nahradíme je správnými hodnotami z tabulky adult (použita funkce najít a nahradit ve VS-code :) ).

In [12]:
def getUniqueValuesFromColumn(df, col):
    values = df[col].tolist()
    uniqueValues = []
    for i in values:
        if(i not in uniqueValues):
            uniqueValues.append(i)
    return uniqueValues

uniqueCountryValues = getUniqueValuesFromColumn(adult, "country")

Před samotným spojením tabulek ještě odstraníme z GDP tabulky ty státy, které nejsou obsaženy v tabulce adult, aby po spojení nedošlo k vytvoření několika řádků s prázdnými hodnotami (vyplněný by byl pouze stát). Použijeme proto další funkci, do které přidáme dataset, název sloupce a pole hodnot států. Funkce nám pak vráti dataset obsahující pouze řádky se státem, který byl obsažený v poskytnutém poli (uniqueCountryValues z části před touto).

In [13]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

gdp = filter_rows_by_values(gdp, "country", uniqueCountryValues)

Poté již spojíme datasety a vznikne nám nový, který obsahuje všechny atributy z datasetu adult a navíc hrubý domácí produkt státu, ze kterého daná osoba pochází:

In [14]:
adultWithGdp = pd.merge(adult, gdp, on='country', how='outer')

Nyní můžeme vytvořit nové atributy v datasetu podle našich potřeb. Následující kód vytvoří nový sloupec `AgeStatus` a připíše hodnotu "Young" lidem mladším než 30, "Middle aged" lidem mezi 30 a 60 a "Old" lidem starším než 60.

In [15]:
adultWithGdp['AgeStatus'] = adultWithGdp.apply(lambda row: "Young" if row.Age < 30 else ("Middle aged" if row.Age >= 30 & row.Age < 60 else "Old"), axis=1)

## Krok 3 - <a href="https://lispminer.vse.cz/wiki/doku.php?id=mkl:start">KL miner</a>
Analytická procedura KL-Miner hledá všechny zajímavé frekvence kombinací hodnot kategorií dvou předzpracovaných atributů, navíc počítané na podmnožinách analyzovaných dat zadaných pomocí booleovských atributů s bohatou syntaxí.

U KL mineru nastavuje základní parametry:
`conf`: Konfidence - pravděpodobnost, že podmínka bude vyplněna
`base`: Základ - počet vyhovujících případů
`minlen, maxlen`: Maximální a minimální délka podmnožin/sekvencí/cutů

V Clever Mineru používáme KL-miner následovně:

In [16]:
from cleverminer import cleverminer

clm = cleverminer(df=adultWithGdp, proc='4ftMiner',
               quantifiers= {'conf':0.67, 'Base':10000},
               ante ={
                    'attributes':[
                        {'name': 'AgeStatus', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
               )

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  4ftMiner
Starting to mine rules.
Done. Total verifications : 1, rules 1,control number:0, times: prep 3.314272403717041, processing 0.0


V předchozí ukázce jako antecedent nastavujeme vytvořený atribut `AgeStatus` a sukcedent `income`. Konfidenci jsme nastavili na 67% a základ 10 000.

Přepisem do lidsky srozumitelného jazyka toto znamená: Jestliže si vezmeme nějakou hodnotu z `AgeStatus`(Old/Yound/Middle aged), je v nějakém případě pravděpodobnost vyšší než 67%, že v tom případě daná osoba bude vydělávat více či méně než 50K?

Výsledek vypíšeme následovně:

In [ ]:
clm.print_summary()
clm.print_rulelist()
clm.print_rule(1)

Jako výstup uvidíme toto:

Cedents:

  antecedent : AgeStatus(Middle aged)

  succcedent : income(<=50K)

![lol](./img/4table.png)

Z tabulky vidíme, že Clever Miner skutečně našel takový vztah mezi `AgeStatus` a `income`. Konkrétně z výsledku můžeme vyvodit toto pravidlo: jestliže je někdo middle aged (30-60), pak je šance minimálně 67%, že bude mít plat menší nebo rovno 50 tisíc dolarů za rok a zároveň takových lidí je viíce než 10 000. V kódu je uvedeno více příkladů s prací s KL minerem.